# Noncommutative variables

We consider the Example 2.11 of [BKP16] in which the polynomial with noncommutative variables
$(x * y + x^2)^2 = x^4 + x^3y + xyx^2 + xyxy$ is tested to be sum-of-squares.

[BKP16] Sabine Burgdorf, Igor Klep, and Janez Povh.
*Optimization of polynomials in non-commuting variables*.
Berlin: Springer, 2016.

In [1]:
using DynamicPolynomials
@ncpolyvar x y
p = (x * y + x^2) * (x * y + x^2)

x⁴ + x³y + xyx² + xyxy

In [2]:
import CSDP
using JuMP
optimizer_constructor = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)

MathOptInterface.OptimizerWithAttributes(CSDP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.Silent() => true])

The Newton polytope method has not been adapted to the noncommutative case yet,
so we force the choice of certificate to `MaxDegree` instead of `Newton`.

In [3]:
using SumOfSquares
model = Model(optimizer_constructor)
con_ref = @constraint(model, p in SOSCone(),
    certificate = SumOfSquares.Certificate.MaxDegree(SOSCone(), MonomialBasis, 4))

(1)x⁴ + (1)x³y + (1)xyx² + (1)xyxy is SOS

In [4]:
optimize!(model)

We see that both the monomials `xy` and `yx` are considered separately, this is a difference with the commutative version.

In [5]:
certificate_basis(con_ref)

MonomialBasis{Monomial{false},MonomialVector{false}}(Monomial{false}[x², xy, y², yx, x, y, 1])

We see that the solution correctly uses the monomial `xy` instead of `yx`. We also identify that only the monomials `x^2` and `xy` would be needed. This would be dectected by the Newton chip method of [Section 2.3, BKP16].

In [6]:
gram_matrix(con_ref).Q

7×7 SymMatrix{Float64}:
 1.0  1.0  0.0         0.0         0.0         0.0         0.0       
 1.0  1.0  0.0         0.0         0.0         0.0         0.0       
 0.0  0.0  9.02072e-9  0.0         0.0         0.0         0.0       
 0.0  0.0  0.0         9.02072e-9  0.0         0.0         0.0       
 0.0  0.0  0.0         0.0         9.02072e-9  0.0         0.0       
 0.0  0.0  0.0         0.0         0.0         9.02072e-9  0.0       
 0.0  0.0  0.0         0.0         0.0         0.0         9.02072e-9

When asking for the SOS decomposition, the numerically small entries makes the solution less readable.

In [7]:
sos_decomposition(con_ref)

(-0.9999999979815454*x^2 - 1.0000000006339345*x*y)^2 + (-7.164435590170406e-5*x^2 + 7.164435571167535e-5*x*y)^2 + (9.49774622097805e-5*x)^2 + (9.49774622097805e-5*y)^2 + (9.49774622097805e-5)^2 + (9.49774622097805e-5*y*x)^2 + (9.49774622097805e-5*y^2)^2

They are however easily discarded by using a nonzero tolerance:

In [8]:
sos_decomposition(con_ref, 1e-6)

(-0.9999999979815454*x^2 - 1.0000000006339345*x*y)^2